In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot_prediction_error(df_list, label_list, title, ymax, xmax):
    # Theme
    plt.figure(figsize=(16, 8))
    sns.set_theme(style="whitegrid")
    sns.set_context("paper", font_scale=3.5)
    # Data
    for df, label in zip(df_list, label_list):
        ax = sns.lineplot(data=df, x="tau", y="value", label=label, linewidth=3)
    # Axes plot
    ax.set(xlabel="tau", ylabel="NSSE")
    ax.set(ylim=(-0.001, ymax), xlim=(0, xmax))
    ax.legend()
    ax.set_title(title)